In [26]:
import ast, inspect 
import numpy as np
import pype
from hemul.ciphertext import Ciphertext
from modifier import NumpyReplacer, RescaleAdder, BinOpReplacer

# Varaince Example

In pandas, calculating variance is as easy as...

```python
result = df['people'].var()

```

# If user writes it manually

In [30]:
#@pype.jit
def var(data:Ciphertext):
    """Clear text version.
    """
    m = np.mean(data)
    diff = (data - m)
    result = np.sum(diff*diff)/len(data)
    return result

# Ordinary
arr = np.array([1,2,3,4,5,6,7,8,9,10])
var(arr)

8.25

In [28]:
@pype.jit  # <------ envoke JIT compiler 
def var(data:Ciphertext):
    """Clear text version.
    """
    m = np.mean(data)
    diff = (data - m)
    result = np.sum(diff*diff)/len(data)
    return result

### FHE
ctxt = encryptor.encrypt(arr)
res = var(ctxt)

[static_analyzer] Inferencing data types... 
Found a numpy call np mean
Found a numpy call np sum
entering Call
Found a numpy call algo fhe_mean
entering Call
Found a numpy call algo fhe_sum_reduce
entering Call
Found a call len skipping...
leaving Call


AttributeError: 'Assign' object has no attribute 'value'

In [29]:
print(decryptor.decrypt(res)[0])

8.2499832


# Explaination

### Parse the code and build AST

In [ ]:
source_code = inspect(var)

1. replace np.mean() and np.sum() -> algo.fhe_mean() and algo.fhe_reduce_sum()  
2. identify +, -, *, and / operations

In [31]:
tree = ast.parse(inspect.getsource(var))
print(ast.dump(tree, indent=2))

Module(
  body=[
    FunctionDef(
      name='var',
      args=arguments(
        posonlyargs=[],
        args=[
          arg(
            arg='data',
            annotation=Name(id='Ciphertext', ctx=Load()))],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
      body=[
        Expr(
          value=Constant(value='Clear text version.\n    ')),
        Assign(
          targets=[
            Name(id='m', ctx=Store())],
          value=Call(
            func=Attribute(
              value=Name(id='np', ctx=Load()),
              attr='mean',
              ctx=Load()),
            args=[
              Name(id='data', ctx=Load())],
            keywords=[])),
        Assign(
          targets=[
            Name(id='diff', ctx=Store())],
          value=BinOp(
            left=Name(id='data', ctx=Load()),
            op=Sub(),
            right=Name(id='m', ctx=Load()))),
        Assign(
          targets=[
            Name(id='result', ctx=Store())],
          value=

In [13]:
print(ast.unparse(tree))

def var(data: Ciphertext):
    """Clear text version.
    """
    m = np.mean(data)
    diff = data - m
    result = np.sum(diff * diff) / len(data)
    return result


In [14]:
def is_op_cc(node:ast.BinOp):
    lhs = node.left
    rhs = node.right
    #
    # How do I know the data type without running the code?
    # have a look at pypy, numba, or mypy
    

In [15]:
class RescaleAfter_assignr(ast.NodeTransformer):
    #self._expr_statement = False

    def visit_Assign(self, node):
        self._expr_statement = True
        self.generic_visit(node)
        self._expr_statement = False    

In [32]:
tree = ast.parse(inspect.getsource(var))

visitor = NumpyReplacer()
visitor.visit(tree)

tree = ast.fix_missing_locations(tree)
print(ast.dump(tree, indent=2))
print("------")
print(ast.unparse(tree))

entering Call
Found a numpy call np mean
leaving Call
entering Call
Found a numpy call np sum
leaving Call
entering Call
Found a call len skipping...
leaving Call
Module(
  body=[
    FunctionDef(
      name='var',
      args=arguments(
        posonlyargs=[],
        args=[
          arg(
            arg='data',
            annotation=Name(id='Ciphertext', ctx=Load()))],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
      body=[
        Expr(
          value=Constant(value='Clear text version.\n    ')),
        Assign(
          targets=[
            Name(id='m', ctx=Store())],
          value=Call(
            func=Attribute(
              value=Name(id='algo', ctx=Load()),
              attr='fhe_mean',
              ctx=Load()),
            args=[
              Name(id='data', ctx=Load())],
            keywords=[])),
        Assign(
          targets=[
            Name(id='diff', ctx=Store())],
          value=BinOp(
            left=Name(id='data', ctx=Load(

In [23]:
BinOpReplacer().visit(tree)
tree = ast.fix_missing_locations(tree)
print(ast.unparse(tree))

This is sub
leaving BinOp
This is mult
leaving BinOp
def var(data: Ciphertext):
    """Clear text version.
    """
    m = algo.fhe_mean(data)
    diff = ev.sub(data, m)
    result = algo.fhe_sum_reduce(ev.mult(diff, diff)) / len(data)
    return result


In [24]:
print(ast.dump(tree, indent=2))

Module(
  body=[
    FunctionDef(
      name='var',
      args=arguments(
        posonlyargs=[],
        args=[
          arg(
            arg='data',
            annotation=Name(id='Ciphertext', ctx=Load()))],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
      body=[
        Expr(
          value=Constant(value='Clear text version.\n    ')),
        Assign(
          targets=[
            Name(id='m', ctx=Store())],
          value=Call(
            func=Attribute(
              value=Name(id='algo', ctx=Load()),
              attr='fhe_mean',
              ctx=Load()),
            args=[
              Name(id='data', ctx=Load())],
            keywords=[])),
        Assign(
          targets=[
            Name(id='diff', ctx=Store())],
          value=Call(
            func=Attribute(
              value=Name(id='ev', ctx=Load()),
              attr='sub',
              ctx=Load()),
            args=[
              Name(id='data', ctx=Load()),
              Na

In [ ]:
bb = tree.body[0].body[2].value

In [ ]:
def fhe_sum_reduce(ctxt):
    """np.sum() -> sum_reduce()"""
    ctxt_ = ev.copy(ctxt)
    for i in range(int(np.log2(len(ctxt)))):
        tmp = ev.copy(ctxt_)
        ev.lrot(tmp, 2**i, inplace=True)
        ev.add(ctxt_, tmp, inplace=True)   
    return ctxt_

def fhe_mean(ctxt):
    """np.mean() -> mean()"""
    n = algo.encode_repeat(ctxt._n_elements)
    summed = fhe_sum_reduce(ctxt)
    return ev.div_by_plain(summed, n)


In [79]:
def fhe_var(ctxt):
    ctxt_mean = fhe_mean(ctxt)
    ev.rescale_next(ctxt_mean)
    ctxt_mean = algo.put_mask(ctxt_mean, np.arange(ctxt._n_elements))
    ev.rescale_next(ctxt_mean)
    ev.mod_down_to(ctxt, ctxt_mean.logq)
    sub = ev.sub(ctxt, ctxt_mean)
    squared = ev.mult(sub, sub) 
    ev.rescale_next(squared)
    summed_eq = fhe_sum_reduce(squared)

    n = encoder.encode([len(data)]*ctxt.nslots) 
    res = ev.div_by_plain(summed_eq, n)
    return res

In [99]:
org="""Module(
  body=[
    FunctionDef(
      name='var',
      args=arguments(
        posonlyargs=[],
        args=[
          arg(arg='data')],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
      body=[
        Expr(
          value=Constant(value='Clear text version')),
        Assign(
          targets=[
            Name(id='m', ctx=Store())],
          value=Call(
            func=Attribute(
              value=Name(id='np', ctx=Load()),
              attr='mean',
              ctx=Load()),
            args=[
              Name(id='data', ctx=Load())],
            keywords=[])),
        Assign(
          targets=[
            Name(id='diff', ctx=Store())],
          value=BinOp(
            left=Name(id='data', ctx=Load()),
            op=Sub(),
            right=Name(id='m', ctx=Load()))),
        Assign(
          targets=[
            Name(id='result', ctx=Store())],
          value=BinOp(
            left=Call(
              func=Attribute(
                value=Name(id='np', ctx=Load()),
                attr='sum',
                ctx=Load()),
              args=[
                BinOp(
                  left=Name(id='diff', ctx=Load()),
                  op=Mult(),
                  right=Name(id='diff', ctx=Load()))],
              keywords=[]),
            op=Div(),
            right=Call(
              func=Name(id='len', ctx=Load()),
              args=[
                Name(id='data', ctx=Load())],
              keywords=[]))),
        Return(
          value=Name(id='result', ctx=Load()))],
      decorator_list=[])],
  type_ignores=[])"""

modified = """Module(
  body=[
    FunctionDef(
      name='fhe_var',
      args=arguments(
        posonlyargs=[],
        args=[
          arg(arg='ctxt')],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
      body=[
        Expr(
          value=Constant(value='Auto-generated')),
        Assign(
          targets=[
            Name(id='m', ctx=Store())],
          value=Call(
            func=Attribute(
              value=Name(id='algo', ctx=Load()),
              attr='fhe_mean',
              ctx=Load()),
            args=[
              Name(id='ctxt', ctx=Load())],
            keywords=[])),
        Assign(
          targets=[
            Name(id='diff', ctx=Store())],
          value=BinOp(
            left=Name(id='ctxt', ctx=Load()),
            op=Sub(),
            right=Name(id='m', ctx=Load()))),
        Assign(
          targets=[
            Name(id='result', ctx=Store())],
          value=BinOp(
            left=Call(
              func=Attribute(
                value=Name(id='algo', ctx=Load()),
                attr='fhe_sum_reduce',
                ctx=Load()),
              args=[
                BinOp(
                  left=Name(id='diff', ctx=Load()),
                  op=Mult(),
                  right=Name(id='diff', ctx=Load()))],
              keywords=[]),
            op=Div(),
            right=Call(
              func=Name(id='len', ctx=Load()),
              args=[
                Name(id='ctxt', ctx=Load())],
              keywords=[]))),
        Return(
          value=Name(id='result', ctx=Load()))],
      decorator_list=[])],
  type_ignores=[])"""

In [94]:
from ast import (Module, FunctionDef, arguments, arg, 
                Expr, Constant, Assign, Name, Store, 
                Call, Attribute, Load, BinOp, Sub, Mult, Div, Return)

fhet = eval(modified)

In [100]:
normal = eval(org)

In [101]:
normal.

In [114]:
exe = eval(modified)

In [110]:
ast.unparse(fhet)

"Module(body=[FunctionDef(name='fhe_var', args=arguments(posonlyargs=[], args=[arg(arg='ctxt')], kwonlyargs=[], kw_defaults=[], defaults=[]), body=[Expr(value=Constant(value='Auto-generated')), Assign(targets=[Name(id='m', ctx=Store())], value=Call(func=Attribute(value=Name(id='algo', ctx=Load()), attr='fhe_mean', ctx=Load()), args=[Name(id='ctxt', ctx=Load())], keywords=[])), Assign(targets=[Name(id='diff', ctx=Store())], value=BinOp(left=Name(id='ctxt', ctx=Load()), op=Sub(), right=Name(id='m', ctx=Load()))), Assign(targets=[Name(id='result', ctx=Store())], value=BinOp(left=Call(func=Attribute(value=Name(id='algo', ctx=Load()), attr='fhe_sum_reduce', ctx=Load()), args=[BinOp(left=Name(id='diff', ctx=Load()), op=Mult(), right=Name(id='diff', ctx=Load()))], keywords=[]), op=Div(), right=Call(func=Name(id='len', ctx=Load()), args=[Name(id='ctxt', ctx=Load())], keywords=[]))), Return(value=Name(id='result', ctx=Load()))], decorator_list=[])], type_ignores=[])"

In [102]:
fhet = ast.parse(modified)

In [107]:
print(ast.unparse(tree))

def var(data):
    """Clear text version
    """
    m = np.mean(data)
    diff = data - m
    result = np.sum(diff * diff) / len(data)
    return result


In [105]:
ast.unparse(fhet.body)

"Module(body=[FunctionDef(name='fhe_var', args=arguments(posonlyargs=[], args=[arg(arg='ctxt')], kwonlyargs=[], kw_defaults=[], defaults=[]), body=[Expr(value=Constant(value='Auto-generated')), Assign(targets=[Name(id='m', ctx=Store())], value=Call(func=Attribute(value=Name(id='algo', ctx=Load()), attr='fhe_mean', ctx=Load()), args=[Name(id='ctxt', ctx=Load())], keywords=[])), Assign(targets=[Name(id='diff', ctx=Store())], value=BinOp(left=Name(id='ctxt', ctx=Load()), op=Sub(), right=Name(id='m', ctx=Load()))), Assign(targets=[Name(id='result', ctx=Store())], value=BinOp(left=Call(func=Attribute(value=Name(id='algo', ctx=Load()), attr='fhe_sum_reduce', ctx=Load()), args=[BinOp(left=Name(id='diff', ctx=Load()), op=Mult(), right=Name(id='diff', ctx=Load()))], keywords=[]), op=Div(), right=Call(func=Name(id='len', ctx=Load()), args=[Name(id='ctxt', ctx=Load())], keywords=[]))), Return(value=Name(id='result', ctx=Load()))], decorator_list=[])], type_ignores=[])"

In [ ]:
ast.

In [92]:
# Converted version
print(ast.unparse(fhet))

Module(body=[FunctionDef(name='fhe_var', args=arguments(posonlyargs=[], args=[arg(arg='ctxt')], kwonlyargs=[], kw_defaults=[], defaults=[]), body=[Expr(value=Constant(value='Auto-generated')), Assign(targets=[Name(id='m', ctx=Store())], value=Call(func=Attribute(value=Name(id='algo', ctx=Load()), attr='fhe_mean', ctx=Load()), args=[Name(id='ctxt', ctx=Load())], keywords=[])), Assign(targets=[Name(id='diff', ctx=Store())], value=BinOp(left=Name(id='ctxt', ctx=Load()), op=Sub(), right=Name(id='m', ctx=Load()))), Assign(targets=[Name(id='result', ctx=Store())], value=BinOp(left=Call(func=Attribute(value=Name(id='algo', ctx=Load()), attr='fhe_sum_reduce', ctx=Load()), args=[BinOp(left=Name(id='diff', ctx=Load()), op=Mult(), right=Name(id='diff', ctx=Load()))], keywords=[]), op=Div(), right=Call(func=Name(id='len', ctx=Load()), args=[Name(id='ctxt', ctx=Load())], keywords=[]))), Return(value=Name(id='result', ctx=Load()))], decorator_list=[])], type_ignores=[])


In [ ]:
if op_cc(a,b):
    # Match scale
    if a.logp > b.logp:
        add_rescale_a_b_before()
    elif a.logp < b.logp:
        add_rescale_b_a_before()
    # Match level
    if a.logp > b.logp:
        add_modswitch_a_b_before()
    elif a.logp < b.logp:
        add_modswitch_b_a_before()

In [ ]:
def add_rescale_b_a_before():
    pass

def add_rescale_a_b_before():
    pass

def add_modswitch_b_a_before():
    pass

def add_modswitch_a_b_before():
    pass
    
def op_cp(a,b):
    return isinstance(a, Ciphertext) and if isinstance(b, Plaintext)
    
def op_cc(a,b):
    return isinstance(a, Ciphertext) and if isinstance(b, Ciphertext)